Installing dependencies

In [16]:
%%script echo skipping
# %pip install binance-connector
%pip install tradingview-ta
%pip install pandas
%pip install python-dotenv
%pip install python-binance

skipping


Trying out TradingView API

In [17]:
%%script echo skipping
from tradingview_ta import TA_Handler, Interval, Exchange

ADA = TA_Handler(
    symbol="ADAUSDT",
    screener="crypto",
    exchange="BINANCE",
    interval=Interval.INTERVAL_1_DAY,
    # proxies={'http': 'http://example.com:8080'} # Uncomment to enable proxy (replace the URL).
)
print(ADA.get_analysis().summary)
# Example output: {"RECOMMENDATION": "BUY", "BUY": 8, "NEUTRAL": 6, "SELL": 3}

skipping


In [18]:
from datetime import datetime
import os

import numpy as np
import pandas as pd
from binance.client import Client

In [19]:
from dotenv import load_dotenv
load_dotenv()

api_key = os.environ['BINANCE_API_KEY']
api_secret = os.environ['BINANCE_PASSWORD']

# client = Client(api_key, api_secret)
# print('Using Binance TestNet Server')
# print('')
# print(binance_client.get_account())
# client = Client(base_url='https://testnet.binance.vision')
# print(client.time())
# print(client.account())

# from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret, testnet=True)
# client.ping()
# client.get_server_time()
info = client.get_symbol_info('ETHUSDT')
print(info)

{'symbol': 'ETHUSDT', 'status': 'TRADING', 'baseAsset': 'ETH', 'baseAssetPrecision': 8, 'quoteAsset': 'USDT', 'quotePrecision': 8, 'quoteAssetPrecision': 8, 'baseCommissionPrecision': 8, 'quoteCommissionPrecision': 8, 'orderTypes': ['LIMIT', 'LIMIT_MAKER', 'MARKET', 'STOP_LOSS_LIMIT', 'TAKE_PROFIT_LIMIT'], 'icebergAllowed': True, 'ocoAllowed': True, 'quoteOrderQtyMarketAllowed': True, 'allowTrailingStop': True, 'cancelReplaceAllowed': True, 'isSpotTradingAllowed': True, 'isMarginTradingAllowed': False, 'filters': [{'filterType': 'PRICE_FILTER', 'minPrice': '0.01000000', 'maxPrice': '100000.00000000', 'tickSize': '0.01000000'}, {'filterType': 'PERCENT_PRICE', 'multiplierUp': '5', 'multiplierDown': '0.2', 'avgPriceMins': 1}, {'filterType': 'LOT_SIZE', 'minQty': '0.00001000', 'maxQty': '9000.00000000', 'stepSize': '0.00001000'}, {'filterType': 'MIN_NOTIONAL', 'minNotional': '10.00000000', 'applyToMarket': True, 'avgPriceMins': 1}, {'filterType': 'ICEBERG_PARTS', 'limit': 10}, {'filterType

In [20]:
def get_data_frame(symbol):
    starttime = '3 days ago UTC'
    interval = '15m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])

    return df

get_data_frame('ETHUSDT')

,date,open,high,low,close
0,1667045700000,1614.73000000,1626.46000000,1229.00000000,1621.98000000
1,1667046600000,1621.99000000,1626.02000000,1619.25000000,1624.53000000
2,1667047500000,1624.49000000,1626.68000000,1620.66000000,1626.25000000
3,1667048400000,1625.91000000,1626.68000000,1229.00000000,1622.26000000
4,1667049300000,1622.19000000,3287.09000000,1608.69000000,1617.85000000
...,...,...,...,...,...
283,1667300400000,1606.50000000,1615.37000000,1200.00000000,1593.28000000
284,1667301300000,1593.29000000,2500.00000000,1200.00000000,1591.72000000
285,1667302200000,1591.60000000,1594.64000000,1588.68000000,1590.65000000
286,1667303100000,1590.03000000,1593.85000000,1589.40000000,1590.66000000


In [21]:
def macd_logic(symbol):

    symbol_df = get_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['200_ema'] < symbol_df['close'])), 1, 0)

    return symbol_df

macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
0,2022-10-29 12:15:00,1614.73,1626.46,1229.00,1621.98,0.000000,0.000000,1621.980000,0
1,2022-10-29 12:30:00,1621.99,1626.02,1619.25,1624.53,0.203419,0.040684,1622.005373,1
2,2022-10-29 12:45:00,1624.49,1626.68,1620.66,1626.25,0.497682,0.132083,1622.047608,1
3,2022-10-29 13:00:00,1625.91,1626.68,1229.00,1622.26,0.404268,0.186520,1622.049722,1
4,2022-10-29 13:15:00,1622.19,3287.09,1608.69,1617.85,-0.025321,0.144152,1622.007933,0
...,...,...,...,...,...,...,...,...,...
283,2022-11-01 11:00:00,1606.50,1615.37,1200.00,1593.28,4.345218,-23.504807,1568.331961,1
284,2022-11-01 11:15:00,1593.29,2500.00,1200.00,1591.72,10.656694,-16.672506,1568.564678,1
285,2022-11-01 11:30:00,1591.60,1594.64,1588.68,1590.65,15.394783,-10.259049,1568.784432,1
286,2022-11-01 11:45:00,1590.03,1593.85,1589.40,1590.66,18.932319,-4.420775,1569.002099,1


# BACKTESTING

Backtesting completed on 6 month historical data on 15m time frame

In [22]:
def get_six_month_data_frame(symbol):
    starttime = '6 months ago UTC'
    interval = '15m'
    data = client.get_historical_klines(symbol, interval, starttime)

    for row in data:
        del row[5:]
    
    df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close'])
    return df

six_month_eth_df_raw = get_six_month_data_frame('ETHUSDT')

In [23]:
six_month_eth_df_raw

,date,open,high,low,close
0,1664969400000,1334.55000000,1334.55000000,1334.20000000,1334.20000000
1,1664970300000,1334.00000000,1334.00000000,1331.21000000,1331.21000000
2,1664971200000,1334.19000000,4277.48000000,1069.37000000,4277.48000000
3,1664972100000,4277.48000000,4277.48000000,4277.48000000,4277.48000000
4,1664973000000,1331.64000000,4277.48000000,1328.61000000,1335.00000000
...,...,...,...,...,...
2590,1667300400000,1606.50000000,1615.37000000,1200.00000000,1593.28000000
2591,1667301300000,1593.29000000,2500.00000000,1200.00000000,1591.72000000
2592,1667302200000,1591.60000000,1594.64000000,1588.68000000,1590.65000000
2593,1667303100000,1590.03000000,1593.85000000,1589.40000000,1590.66000000


In [30]:
def test_macd_logic(symbol):

    symbol_df = get_six_month_data_frame(symbol)

    symbol_df['open'] = pd.to_numeric(symbol_df['open'])
    symbol_df['high'] = pd.to_numeric(symbol_df['high'])
    symbol_df['low'] = pd.to_numeric(symbol_df['low'])
    symbol_df['close'] = pd.to_numeric(symbol_df['close'])
    symbol_df['date'] = pd.to_datetime(symbol_df['date'], unit = 'ms')


    short_ema = symbol_df['close'].ewm(span = 12, adjust = False).mean()
    long_ema = symbol_df['close'].ewm(span = 26, adjust = False).mean()
    ema_200 = symbol_df['close'].ewm(span = 200, adjust = False).mean()

    macd = short_ema - long_ema
    signal = macd.ewm(span = 9, adjust = False).mean()

    symbol_df['macd'] = macd
    symbol_df['signal'] = signal
    symbol_df['200_ema'] = ema_200

    # print(symbol_df.info())


    symbol_df['Trigger'] = np.where(((symbol_df['macd'] > symbol_df['signal']) & (symbol_df['close'] > symbol_df['200_ema']) & (symbol_df['macd'] < 0) & (symbol_df['signal'] > symbol_df['macd'].shift())), 1, 0)

    symbol_df = symbol_df.iloc[200:]

    triggered_df = symbol_df.loc[symbol_df['Trigger'] == 1]

    return triggered_df

test_macd_logic('ETHUSDT')

,date,open,high,low,close,macd,signal,200_ema,Trigger
699,2022-10-12 18:15:00,1298.34,1298.75,1295.22,1298.69,-0.413746,-0.496235,1295.099320,1
727,2022-10-13 01:15:00,1296.79,1297.91,1295.98,1296.45,-0.345359,-0.358448,1295.597400,1
904,2022-10-14 21:30:00,1229.00,4895.00,900.00,1325.55,-14.413343,-20.324988,1293.788466,1
932,2022-10-15 04:30:00,1229.00,1380.72,1229.00,1284.18,-14.042104,-14.720269,1279.958983,1
978,2022-10-15 16:00:00,1235.90,1235.90,1235.90,1235.90,-87.119528,-91.485081,1224.168719,1
979,2022-10-15 16:15:00,1235.90,1235.90,1235.90,1235.90,-56.838941,-84.555853,1224.285448,1
1063,2022-10-16 13:15:00,1235.90,1256.59,1229.00,1229.00,-32.497905,-36.610793,1214.327007,1
1112,2022-10-17 01:30:00,1263.48,1305.86,1228.99,1242.80,-1.878983,-1.928900,1226.554412,1
1113,2022-10-17 01:45:00,1229.00,1280.60,1229.00,1242.80,-1.031323,-1.749385,1226.716060,1
1248,2022-10-18 11:30:00,1326.59,1329.49,1326.59,1327.55,-1.701504,-1.805296,1300.257806,1
